In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [1]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_feature.parquet'
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/merged_candidates_150_test_feature.parquet'

In [11]:
train_freq_file = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/last2item_freq.csv'
test_freq_file = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates_phase2/last2item_freq_test.csv'

train_freq={}
test_freq={}
with open(train_freq_file, 'r') as f:
    for line in f:
        id, freq = line.strip().split('\t')
        train_freq[id] = int(freq)
with open(test_freq_file, 'r') as f:
    for line in f:
        id, freq = line.strip().split('\t')
        test_freq[id] = int(freq)

In [3]:
merged_candidates_feature = pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

In [4]:
merged_candidates_feature_test = pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [13]:
merged_candidates_feature['last2item_freq'] = merged_candidates_feature['product'].apply(lambda x: train_freq.get(x, 0))
merged_candidates_feature_test['last2item_freq'] = merged_candidates_feature_test['product'].apply(lambda x: train_freq.get(x, 0))
merged_candidates_feature['last2item_freq_test'] = merged_candidates_feature['product'].apply(lambda x: test_freq.get(x, 0))
merged_candidates_feature_test['last2item_freq_test'] = merged_candidates_feature_test['product'].apply(lambda x: test_freq.get(x, 0))

In [ ]:
cast_dtype(merged_candidates_feature, ['last2item_freq', 'last2item_freq_test'])
merged_candidates_feature.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [ ]:
cast_dtype(merged_candidates_feature_test, ['last2item_freq', 'last2item_freq_test'])
merged_candidates_feature_test.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')